In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --only-binary :all: great_expectations

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import great_expectations as gx
# See what context would be created without mode
print(f"Default context type: {type(gx.get_context()).__name__}")


print(gx.__version__)
print(pd.__version__)

Default context type: FileDataContext
0.18.8
2.2.3


In [5]:
# Force ephemeral with debug
try:
    context = gx.get_context(mode="ephemeral")
except Exception as e:
    print(f"Full error: {e}")
    print("Trying manual creation...")
    from great_expectations.data_context import EphemeralDataContext
    context = EphemeralDataContext()

Full error: Provided mode ephemeral returned context of type FileDataContext instead of EphemeralDataContext; please check your input arguments.
Trying manual creation...


TypeError: EphemeralDataContext.__init__() missing 1 required positional argument: 'project_config'

In [ ]:
data_source_name = 'temperature_data'
data_source = context.data_sources.add_pandas(name=data_source_name)

In [ ]:
data_asset_name = 'temperature_entity_asset'
data_asset = data_source.add_dataframe_asset(name = data_asset_name)

In [ ]:
batch_definition_name ='full_batch' 
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

In [ ]:
expectation_temp = gx.expectations.ExpectColumnValuesToBeBetween(column="Temperature",max_value=45,min_value=10)
expectation_city = gx.expectations.ExpectColumnDistinctValuesToBeInSet(column="City",value_set=["Mumbai","Delhi"])

In [ ]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)

In [ ]:
#adding expectations to the Expectation suite

expectation_suite.add_expectation(expectation_temp)
expectation_suite.add_expectation(expectation_city)

In [ ]:
validation_def_name = "temperature_data_validation"
validation_definition_ref = gx.ValidationDefinition(data=batch_definition,suite=expectation_suite,name=validation_def_name)

In [ ]:
validation_definition = context.validations.add(validation_definition_ref)

In [ ]:
data_df = pd.read_csv("temperature_data.csv")
data_df.head()

In [ ]:
batch_parameters = {dataframe=data_df}

In [ ]:
validation_results = validation_definition.run(batch_parameters=batch_parameters)

In [ ]:
print(validation_results)